In [29]:
import pandas as pd 
import numpy as np
from neurocombat_sklearn import CombatModel
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import cross_validate
from neurocombat_sklearn import CombatModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed, parallel_backend
import multiprocessing
import time
import pickle
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import math

### Load the original data

In [30]:
Xs = np.genfromtxt("./data612/Xs.csv", delimiter=',')
Xc = np.genfromtxt("./data612/Xc.csv", delimiter=',')
Xi = np.genfromtxt("./data612/Xi.csv", delimiter=',')

site = np.genfromtxt("./data612/Site.csv", delimiter=',')

Y  = np.genfromtxt("./data612/Y.csv", delimiter=',')

# Preview the size of Xs
print("Xs.shape before attaching site",Xs.shape)
print("Y.shape",Y.shape)
print("site.shape",site.shape)

# append site data to the end of X for simplicity
Xs = np.concatenate((Xs, np.reshape(site, (-1, 1))), axis=1)
Xc = np.concatenate((Xc, np.reshape(site, (-1, 1))), axis=1)
Xi = np.concatenate((Xi, np.reshape(site, (-1, 1))), axis=1)

print("Xs.shape after attaching site",Xs.shape)
print("Xc.shape after attaching site",Xc.shape)
print("Xi.shape after attaching site",Xi.shape)

Xs.shape before attaching site (612, 738)
Y.shape (612,)
site.shape (612,)
Xs.shape after attaching site (612, 739)
Xc.shape after attaching site (612, 56956)
Xi.shape after attaching site (612, 56956)


###  Load analysis results

In [31]:
outer =0
dataset = 'c'
fold = str(outer)

if dataset == 's':
    X_data = Xs
elif dataset == 'c':
    X_data = Xc
elif dataset == 'i':
    X_data = Xi

print(X_data.shape)

(612, 56956)


In [32]:
filtered_good_results = pd.read_pickle("./X"+dataset+"_results/results/fold" + fold + "/filtered_good_results_fold" + fold + ".pkl")

In [33]:
with open('./split_matlab/outer_train_index.pickle', 'rb') as train:
    outer_train_index = pickle.load(train)

with open('./split_matlab/outer_test_index.pickle', 'rb') as test:
    outer_test_index = pickle.load(test)

In [34]:
with open('outer_train_index.pickle', 'rb') as train_test:
    outer_train_index_1 = pickle.load(train_test)

In [35]:
for i in range(5):
    print(np.array_equal(outer_train_index[i], outer_train_index_1[i]))

False
False
False
False
False


In [36]:
filtered_good_results.head()

,PPV,Subjects Correctly Classified,Number of Progressor Subtypes,Number of Stable Subtypes,Number of Features,Classifier,Weighting,Class with High PPV,Stability of High PPV,Number of features mutually selected,Percentage of Mutual Features Selected across all folds,Features selected with highest frequencies across all folds,Frequencies of the features,Indexes of Instances Correctly Classified
0,1.0,5,4,4,13,LR,1,5,5,2,0.153846,"[46615, 9707, 49376, 29022, 42092, 10258, 2065...","[248, 248, 247, 242, 236, 224, 214, 201, 189, ...","[439, 440, 441, 442, 443]"
1,1.0,5,4,4,30,LR,1,5,3,4,0.133333,"[46615, 9707, 29022, 49376, 42092, 26750, 1312...","[248, 248, 248, 248, 247, 247, 245, 244, 231, ...","[439, 440, 441, 442, 443]"
2,1.0,4,4,4,37,LR,1,5,3,3,0.081081,"[46615, 9707, 29022, 49376, 42092, 26750, 1312...","[248, 248, 248, 247, 247, 247, 246, 244, 241, ...","[439, 440, 441, 443]"
3,1.0,5,4,4,37,LR,1,6,59,3,0.081081,"[46615, 9707, 29022, 49376, 42092, 26750, 1312...","[248, 248, 248, 247, 247, 247, 246, 244, 241, ...","[404, 405, 406, 407, 408]"
4,1.0,2,3,4,28,SVM,20,4,3,3,0.107143,"[46615, 9707, 49376, 26750, 42092, 15217, 2902...","[248, 248, 248, 247, 245, 245, 244, 242, 221, ...","[440, 443]"


In [37]:
print(outer_train_index[outer])
print(len(outer_train_index))

[1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263

In [38]:
print(outer_test_index[outer])
print(len(outer_test_index))

[0, 7, 8, 21, 22, 23, 24, 25, 26, 27, 28, 29, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 297, 298, 299, 300, 301]
5


In [39]:
df_results_of_each_param = pd.read_pickle("./X"+dataset+"_results/results/fold" + fold + "/outerfold" + fold + "_results_for_each_param_setting.pkl")
df_results_of_each_fold = pd.read_pickle("./X"+dataset+"_results/results/fold" + fold + "/outerfold" + fold + "_results_for_each_fold.pkl")
good_results = pd.read_pickle("./X"+dataset+"_results/results/fold" + fold + "/outerfold" +  fold + "_good_results.pkl")

In [40]:
print(np.sort(df_results_of_each_param.iloc[0]['test_index']))

[  6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  47  48  49
  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
 116 117 118 119 120 121 122 123 124 125 126 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 295 296 297 298 299 300 30

In [41]:
index_book_pickle = open("./split_matlab/subjects_scans_index.pickle", "rb")
index_book = pickle.load(index_book_pickle)
index_book_pickle.close()  
print(index_book)

{0: [0, 1, 2, 3, 4, 5], 1: [6, 7, 8, 9], 2: [10, 11, 12, 13, 14, 15, 16, 17], 3: [18, 19, 20, 21, 22, 23, 24, 25], 4: [26, 27, 28, 29, 30, 31, 32], 5: [33, 34, 35], 6: [36, 37, 38], 7: [39, 40, 41], 8: [42, 43, 44, 45, 46], 9: [47, 48, 49, 50], 10: [51, 52, 53], 11: [54, 55, 56, 57], 12: [58, 59, 60, 61, 62], 13: [63, 64], 14: [65, 66, 67], 15: [68, 69, 70], 16: [71, 72, 73, 74], 17: [75, 76, 77, 78, 79], 18: [80], 19: [81], 20: [82, 83, 84], 21: [85, 86, 87], 22: [88], 23: [89], 24: [90], 25: [91], 26: [92], 27: [93], 28: [94, 95], 29: [96], 30: [97, 98], 31: [99], 32: [100], 33: [101], 34: [102, 103], 35: [104], 36: [105], 37: [106], 38: [107], 39: [108], 40: [109], 41: [110], 42: [111], 43: [112], 44: [113], 45: [114], 46: [115], 47: [116], 48: [117], 49: [118], 50: [119], 51: [120, 121], 52: [122, 123], 53: [124], 54: [125], 55: [126], 56: [127, 128], 57: [129], 58: [130], 59: [131], 60: [132, 133], 61: [134], 62: [135], 63: [136], 64: [137], 65: [138], 66: [139], 67: [140, 141], 6

In [42]:
train_scan_index = [index_book.get(i) for i in outer_train_index[outer]]
train_scan_index = [item for subl in train_scan_index for item in subl]
print(train_scan_index)

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 27

In [43]:
test_scan_index = [index_book.get(i) for i in outer_test_index[outer]]
test_scan_index = [item for subl in test_scan_index for item in subl]
print(test_scan_index)

[0, 1, 2, 3, 4, 5, 39, 40, 41, 42, 43, 44, 45, 46, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 607, 608, 609, 610, 611]


In [44]:
class Combatstep():
    """A class to wrap the combat step processing, including fitting the training data and
    apply transformation on the test data

        Attributes
        ----------
        debug_mode : boolean
            a boolean variable to indicate if the debugging mode is activated. If activated, 
            the intermediate results will be printed on standard output
    """

    def __init__(self, debug_mode=False):
        """Apply fit_transform on the training data
    
            Arguments
            ---------
                debug_mode: boolean
                    If true, intermediate results will be printed for debugging purposes 
                    at the end of each function call.
        """
        self.model = CombatModel()
        self.debug_mode = debug_mode
        pass
     
    def fit_transform(self, X, y):
        """Apply fit_transform on the training data
            
            This function should be called upon the training data set. The combat model 
            is fitted and transformed. The data is adjusted/harmonized from the effects 
            of site data, sex data, age data and the labels. The discrete covariates includes
            the sex data and labels. The continuous covariates includes the normalized age
            data. 
            
            Arguments
            ---------
                X: numpy.ndarray
                    The training data with site, sex, age data attached.
                y: numpy.ndarray
                    The training data labels.
            Returns
            -------
                X_harmonized
                    The resulting harmonized data, excluding the site, sex, age data.
        """
        # extract site, sex, age data
        site = X[:,-1].reshape(-1,1)
        sex = X[:,-2].reshape(-1,1)
        age = X[:,-3].reshape(-1,1) 
        
        # exclude site, sex, age from from the data for harmonization
        X_raw = X[:,:-3]
        
        # reshape y array for concatenation
        y = y.reshape(-1,1)
        
        # discrete_covariates: sex
        discrete_covariates = sex
        
        # continuous_covariates: age
        continuous_covariates = age
        
        # apply fit_transform on X
        X_harmonized = self.model.fit_transform(X_raw,
                                            site,
                                            discrete_covariates,
                                            continuous_covariates
                                       )
        
        # attach back the age and sex data to the X_harmonized
        age_sex_data = np.concatenate((age, sex), axis=1)
        X_harmonized = np.concatenate((X_harmonized, age_sex_data), axis=1)
#         print("fit transforma X_harmonized.shape", X_harmonized.shape)
        # print all the computed matrxi if debug mode is enabled
        if self.debug_mode == True:
            print("=" * 50 + " combat step fit_transform " + "="*50)
            print("X_init", X)
            print("X_raw",X_raw)
            print("X_harmonized.shape", X_harmonized.shape)
            print("age_sex_data", age_sex_data)
            print("X_harmonized", X_harmonized)
            
        return X_harmonized
        
    def transform(self, X, y):
        """Apply transformation on the testing data based on the fitted combat model.
        
            This function should be called upon the testing data set. The combat model has 
            already been fitted using the training data and the testing data is transformed
            accordingly. The data is adjusted/harmonized from the effects of site data, sex 
            data, age data and the labels. The discrete covariates includes the sex data 
            and labels. The continuous covariates includes the normalized age
            data. 
    
            Arguments
            ---------
                X: numpy.ndarray
                    The testing data with site, sex, age data attached.
                y: numpy.ndarray
                    The testing data labels.
            Returns
            -------
                X_harmonized
                    The resulting harmonized data, excluding the site, sex, age data.
        """
        # extract site, sex, age data
        site = X[:,-1].reshape(-1,1)
        sex = X[:,-2].reshape(-1,1)
        age = X[:,-3].reshape(-1,1) 
        # exclude site, sex, age from from the data for harmonization
        X_raw = X[:,:-3]
        
        # reshape y array for concatenation
        y = y.reshape(-1,1)
        
        # discrete_covariates: sex
        discrete_covariates = sex
        
        # continuous_covariates: age
        continuous_covariates = age
        
        # apply fit_transform on X
        X_harmonized = self.model.transform(X_raw,
                                            site,
                                            discrete_covariates,
                                            continuous_covariates
                                       )
        # attach back the age and sex data to the X_harmonized
        age_sex_data = np.concatenate((age, sex), axis=1)
        
        X_harmonized = np.concatenate((X_harmonized, age_sex_data), axis=1)
#         print("transforma X_harmonized.shape", X_harmonized.shape)
        # print all the computed matrxi if debug mode is enabled
        if self.debug_mode == True:
            print("=" * 50 + " combat step transform " + "="*50)
            print("X_init", X)
            print("X_raw",X_raw)
            print("discrete_covariates", np.reshape(discrete_covariates,(1, -1)))
            print("continuous_covariates", np.reshape(continuous_covariates,(1, -1)))
            print("X_harmonized.shape", X_harmonized.shape)
            print("X_harmonized", X_harmonized)
            
        return X_harmonized


In [45]:
class Clustering():
    """A class to wrap the clustering step

        Attributes
        ----------
        debug_mode : boolean
            a boolean variable to indicate if the debugging mode is activated. If activated, 
            the intermediate results will be printed on standard output
    """

    def __init__(self, debug_mode=False):
        """Apply fit_transform on the training data
    
            Arguments
            ---------
                debug_mode: boolean
                    If true, intermediate results will be printed for debugging purposes 
                    at the end of each function call.
        """
        self.debug_mode = debug_mode
        pass
    
    # setter for number of subclusters for P
    def setNumP(self, numP):
        self.numP = numP
    
    # setter for number of subclusters for S
    def setNumS(self, numS):
        self.numS = numS
    
    # Step1. Separate out P and S 
    def separate(self, X, y):
        """Separate out the progressor and stable instances to be used for clustering later
        
            As the data in is reindexed by the stratifiedKfold. It is important for us
            to save the indexes of the rows before we separate them out and do clustering 
            accordingly. In this function, while the data is splitted into separate lists 
            by class, the original index for each row is saved. After the clustering is done, 
            we will reindex our combined data set according to the saved indexes
    
            Arguments
            ---------
                X: numpy.ndarray
                    The combined data
                y: numpy.ndarray
                    The combined data labels.
            Returns
            -------
                P_X: list
                    Progressor data
                P_y: list
                    Progressor data labels
                S_X: list
                    Stable data
                S_y: list
                    Stable data labels     
        """
        
        # initialize empty lists to store index of P&S instances in the original data
        index_P, index_S = [], []
        
        # initialize lists to store the separated P and S instances
        P_X, P_y, S_X, S_y= [], [], [], []
        # store P and S instances separately and keep track of the indexes in the original data
        
        for i in range(0, y.shape[0]):
            if y[i] == 1:
                S_X.append(X[i])
                S_y.append(y[i])
                index_S.append(i)
            else: 
                P_X.append(X[i])
                P_y.append(y[i])
                index_P.append(i)
                
        # store the indexes as a global attribute for the clasds
        self.index_P = index_P
        self.index_S = index_S
        
        # print all the computed matrxi if debug mode is enabled
        if self.debug_mode == True:
            print("index_P", self.index_P)
            print("index_S", self.index_S)
            
        return np.asarray(P_X), np.asarray(P_y), np.asarray(S_X), np.asarray(S_y)
        
    def clusteringP(self, P_X):
        """Apply Spectral Clustering on Progressor data
        
            SpectralClustering is performed on P class instances. The number of clusters 
            is specified by the global attribute num_P. At the end of the step, the original
            labels will be discarded and the new labels will be attached to the end of the data
            for convenience. The assign_labels for the SpectralClustering is "discrete" and 
            random_state is set to 0
    
            Arguments
            ---------
                P_X: list
                    The progressor data
            Returns
            -------
                P_X_clustered: numpy.ndarray
                    The combined clustered progressor data with new labels attached to the end
        """
        # apply spectral clustering on the data     
        clusteringP = SpectralClustering(n_clusters=self.numP, assign_labels="kmeans", random_state=0).fit(P_X)
        # get the labels returned
        labels = clusteringP.labels_.reshape(-1,1)
        # attach the new labels to the end of the input data
        P_X = np.asarray(P_X)
        P_X_clustered = np.concatenate((P_X, labels), axis=1)
        
        if self.debug_mode == True:
            print("P labels", np.reshape(labels,(1,-1)))
            print("P_X_clustered", P_X_clustered)
            
        return P_X_clustered
    
    def clusteringS(self, S_X):
        """Apply Spectral Clustering on Stable data
        
            SpectralClustering is performed on the S class instances. The number of clusters 
            is specified by the global attribute num_S. At the end of the step, the original
            labels will be discarded and the new labels will be attached to the end of the data
            for convenience. The assign_labels for the SpectralClustering is "discrete" and 
            random_state is set to 0
    
            Arguments
            ---------
                S_X: list
                    The progressor data
            Returns
            -------
                S_X_clustered: numpy.ndarray
                    The combined clustered progressor data with new labels attached to the end
        """
        
        # apply spectral clustering on the data   
        clusteringS = SpectralClustering(n_clusters=self.numS, assign_labels="kmeans", random_state=0).fit(S_X)
        # get the labels returned
        labels = clusteringS.labels_
        # distinguish against the P labels
        labels = labels + self.numP
        S_X = np.asarray(S_X)
        labels = labels.reshape(-1,1)
        # attach the new labels to the end of the input data
        S_X_clustered = np.concatenate((S_X, labels), axis=1)
        
        if self.debug_mode == True:
            print("S labels", np.reshape(labels, (1,-1)))
            print("S_X_clustered", S_X_clustered)
            
        return S_X_clustered
    
    def transform(self, X, y):
        """Apply Spectral Clustering on stable data and progressor data. 
            Step 1. Separate data out into P and S classes. Save the original data index as 
            the data rows will be reindexed for clustering. 
            Step 2. Perform clustering for P class based on num_P. Attach the new labels.
            Step 3. Perform clustering for S class based on num_S. Attach the new labels. 
            Step 4. Combine the newly clustered P class data and S class data. Reindex the 
            data rows according to the saved indexes. 
            Arguments
            ---------
                X: numpy.ndarray
                    The combined data of both P and S instances.
                y: numpy.ndarray
                    The combined data labels.
            Returns
            -------
                X_output: numpy.ndarray
                    The combined data with newly clustered labels attached.
        """
        
        P_X, P_y, S_X, S_y = self.separate(X, y) 
        P_X_clustered = self.clusteringP(P_X)
        S_X_clustered = self.clusteringS(S_X)
        
        P_X_clustered_with_index = np.concatenate((P_X_clustered, (np.array(self.index_P)).reshape(-1,1)), axis = 1)
        S_X_clustered_with_index = np.concatenate((S_X_clustered, (np.array(self.index_S)).reshape(-1,1)), axis = 1)
        
        X_combined_with_index = np.concatenate((P_X_clustered_with_index, S_X_clustered_with_index), axis=0)
        
        X_sorted = X_combined_with_index[np.argsort(X_combined_with_index[:,-1])]

        new_labels = X_sorted[:,-2]
        X_output = X_sorted[:, :-2]

        if self.debug_mode == True:
            print("P_X_clustered_with_index", P_X_clustered_with_index)
            print("S_X_clustered_with_index", S_X_clustered_with_index)
            print("X_sorted", X_sorted)
            print("X_output", X_output)
            print("X_output.labels", X_output[:, -1])

        return X_output, new_labels

In [46]:
def mrmrSelection(dataframe, nFeature, debug_mode=False):
    """Perform MRMR on the passed in data. 

        This function performs feature selection based on the mRMRe feature selection. 
        It utilizes the mRMRe package originally written in R. A python wrapper for R is
        used here. The eMRMRe returns a list of the selected column indices given the 
        number of features to be selected. 

        Arguments
        ---------
            dataframe: pandas.DataFrame
                The dataframe containing clustered data. 
        Returns
        -------
            df_: pandas.DataFrame
                The dataframe with only selected columns.
            labels: pandas.DataFrame
                The dataframe of only the labels
            colum_indices: list
                The list of indices of the selected columns
    """ 
    base = importr('base')
    mr = importr('mRMRe')
    
    pandas2ri.activate()
    # Convert the data into R format
    with localconverter(ro.default_converter + pandas2ri.converter):
        rdataframe = ro.conversion.py2rpy(dataframe)
    mrmrData = mr.mRMR_data(data = (rdataframe))
    solutionCount = 1  # can i change the value for solutionCount???
    selectionEnsemble = mr.mRMR_ensemble("mRMRe.Data",data = mrmrData, target_indices = (dataframe.shape[1]),
          feature_count = nFeature, solution_count = solutionCount)
    colum_indices = column_indices=(mr.solutions(selectionEnsemble)[0]-1)

    df_ = pd.DataFrame()
    for i,v in enumerate(colum_indices):
        df_[i] = dataframe[v[0]]

    labels = dataframe.iloc[:,-1]

    colum_indices = list(np.reshape(colum_indices,(1,-1))[0])

    if debug_mode:
        print("colum_indices", colum_indices)

    return df_, labels, colum_indices

In [47]:
################################################# Statistical Contrast #################################################
def t_test(X, y):
    
    def separate(X, y):
        # Y is the classes (1=stable, 2=progressor)
        P_X, S_X = [], []
        for i in range(X.shape[0]):
            if  y[i] == 1:
                S_X.append(X[i])
            elif y[i] == 2:
                P_X.append(X[i])
        return np.asarray(P_X), np.asarray(S_X)

    def t_test(P, S, y):
        t_vals = []
        for i in range(P.shape[1]):
            t_score, p_val = stats.ttest_ind(P[:,i], S[:,i])
            tuple_ = (t_score, p_val)
            t_vals.append((tuple_, i))
        return t_vals

    def select(X, y, t_vals, percentage=.1):
        total_nfeatures = X.shape[1]
        filtered_nfeatures = math.ceil(total_nfeatures*percentage)
        # print("total_nfeatures: ",total_nfeatures)
        # print("filtered_nfeatures", filtered_nfeatures)
        sorted_t_val = sorted(t_vals, key=lambda tup: tup[0][0], reverse=True)
        selected_indices = [i[1] for i in sorted_t_val[:filtered_nfeatures]]
        sorted_selected_indices = sorted(selected_indices)
        X_reduced = X[:,sorted_selected_indices]
        return X_reduced, sorted_selected_indices
    
    P_X, S_X = separate(X, y)
    t_vals = t_test(P_X, S_X, y)
    X_reduced, selected_indices = select(X, y, t_vals)
    return X_reduced, selected_indices

In [48]:
def computeSampleWeight(y_train_selected, num_P, weight):
    sample_weight = np.ones(len(y_train_selected))
    range_P = np.arange(num_P)
    for i,v in enumerate(y_train_selected):
        if v in range_P:
            sample_weight[i] = weight * sample_weight[i]
    return sample_weight

In [49]:
def apply(X_train, y_train, X_test, y_test, num_P, num_S, nFeatures, classifier, weighting, column_indices, dataset):
    combat_step = Combatstep()
    X_train_harmonized = combat_step.fit_transform(X_train, y_train)
    X_test_harmonized = combat_step.transform(X_test, y_test)
    
    if dataset=='c' or dataset=='i':
    
        train_sex = X_train_harmonized[:,-1].reshape(-1,1)
        train_age = X_train_harmonized[:,-2].reshape(-1,1)
        train_age_sex_data = np.concatenate((train_age, train_sex), axis=1)
        
        X_train_harmonized_without_age_sex = X_train_harmonized[:,:-2]
        
        X_train_ttest_without_age_sex, columns_ttest = t_test(X_train_harmonized_without_age_sex, y_train)
        X_train_ttest_with_age_sex = np.concatenate((X_train_ttest_without_age_sex, train_age_sex_data), axis=1)
    
        test_sex = X_test_harmonized[:,-1].reshape(-1, 1)
        test_age = X_test_harmonized[:,-2].reshape(-1, 1)
        test_age_sex_data = np.concatenate((test_age, test_sex), axis=1)
        
        X_test_harmonized_without_age_sex = X_test_harmonized[:,:-2]
        X_test_ttest_without_age_sex = X_test_harmonized_without_age_sex[:,columns_ttest]
        X_test_ttest_with_age_sex = np.concatenate((X_test_ttest_without_age_sex, test_age_sex_data), axis=1)
    
    else:
        X_train_ttest_with_age_sex = X_train_harmonized
        X_test_ttest_with_age_sex = X_test_harmonized
    
    integrated_clustering = Clustering()
    integrated_clustering.setNumP(num_P)
    integrated_clustering.setNumS(num_S)
    
    X_train_clustered, y_train_new = integrated_clustering.transform(X_train_ttest_with_age_sex, y_train)
    
    y_train_selected = y_train_new

    X_test_combined = np.concatenate(
        (X_test_harmonized, np.reshape(y_test, (-1, 1))), axis=1)
    
    if dataset!='s':
        check = True
        for item in column_indices:
            if item not in columns_ttest:
                check = False
#         check = all(item in columns_ttest for item in column_indices)

    else:
        check = True
        
    if check != True:
        print("xxxxxxxxxxxxxx === check === xxxxxxxxxxxxxxxx" ,check)
        raise Exception("Feature selection not matched!")
    
    y_test_selected = X_test_combined[:, -1]
    test_sex = X_test_combined[:,-2].reshape(-1, 1)
    test_age = X_test_combined[:,-3].reshape(-1, 1)
    
    
    X_train_harmonized_without_age_sex = X_train_harmonized[:,:-2]
    train_sex = X_train_harmonized[:,-1].reshape(-1,1)
    train_age = X_train_harmonized[:,-2].reshape(-1,1)
    train_age_sex_data = np.concatenate((train_age, train_sex), axis=1)
    
    X_train_selected = X_train_harmonized_without_age_sex[:,column_indices]
    X_train_selected_with_age_sex = np.concatenate((X_train_selected, train_age_sex_data), axis=1)

    y_test_selected = y_test
    X_test_selected = X_test_harmonized[:,column_indices]
    X_test_selected_with_age_sex = np.concatenate((X_test_selected, test_age_sex_data), axis=1)    
    
    sample_weights = computeSampleWeight(y_train_selected, num_P, weighting)
    
    # classification step
    if classifier == 0:
        clf = SVC()
        clf.fit(X_train_selected_with_age_sex, y_train_selected, sample_weight=sample_weights)
        pred_y = clf.predict(X_test_selected_with_age_sex)
        return num_P, num_S, nFeatures, classifier, weighting, pred_y, y_test_selected
    
    elif classifier == 1:
        clf = RandomForestClassifier(random_state=0)
        clf.fit(X_train_selected_with_age_sex, y_train_selected,  sample_weight=sample_weights)
        pred_y = clf.predict(X_test_selected_with_age_sex)
        return num_P, num_S, nFeatures, classifier, weighting, pred_y, y_test_selected
    
    else: 
        clf = LogisticRegression(random_state=0, max_iter=5000)
        clf.fit(X_train_selected_with_age_sex, y_train_selected,  sample_weight=sample_weights)
        pred_y = clf.predict(X_test_selected_with_age_sex)
        return num_P, num_S, nFeatures, classifier, weighting, pred_y, y_test_selected

In [50]:
# def apply(X_train, y_train, X_test, y_test, num_P, num_S, nFeatures, classifier, weighting, column_indices):
#     combat_step = Combatstep()
#     X_train_harmonized = combat_step.fit_transform(X_train, y_train)
#     X_test_harmonized = combat_step.transform(X_test, y_test)
    
#     train_sex = X_train_harmonized[:,-1].reshape(-1,1)
#     train_age = X_train_harmonized[:,-2].reshape(-1,1)
#     train_age_sex_data = np.concatenate((train_age, train_sex), axis=1)
    
#     test_sex = X_test_harmonized[:,-1].reshape(-1, 1)
#     test_age = X_test_harmonized[:,-2].reshape(-1, 1)
#     test_age_sex_data = np.concatenate((test_age, test_sex), axis=1)
    
#     integrated_clustering = Clustering()
#     integrated_clustering.setNumP(num_P)
#     integrated_clustering.setNumS(num_S)
    
#     X_train_clustered, y_train_new = integrated_clustering.transform(X_train_harmonized, y_train)
#     # X_test_combined = np.concatenate((X_test_harmonized, np.reshape(y_test,(-1,1))), axis=1)
# #     print("X_train_clustered.shape", X_train_clustered.shape)
#     y_train_selected = y_train_new
#     X_train_selected = X_train_clustered[:,column_indices]
    
#     X_train_selected_with_age_sex = np.concatenate((X_train_selected, train_age_sex_data), axis=1)

#     y_test_selected = y_test
#     X_test_selected = X_test_harmonized[:,column_indices]
    
#     X_test_selected_with_age_sex = np.concatenate((X_test_selected, test_age_sex_data), axis=1)    
    
#     sample_weights = computeSampleWeight(y_train_selected, num_P, weighting)
    
#     # classification step
#     if classifier == 0:
#         clf = SVC()
#         clf.fit(X_train_selected_with_age_sex, y_train_selected, sample_weight=sample_weights)
#         pred_y = clf.predict(X_test_selected_with_age_sex)
#         return num_P, num_S, nFeatures, classifier, weighting, pred_y, y_test_selected
    
#     elif classifier == 1:
#         clf = RandomForestClassifier(random_state=0)
#         clf.fit(X_train_selected_with_age_sex, y_train_selected,  sample_weight=sample_weights)
#         pred_y = clf.predict(X_test_selected_with_age_sex)
#         return num_P, num_S, nFeatures, classifier, weighting, pred_y, y_test_selected
    
#     else: 
#         clf = LogisticRegression(random_state=0, max_iter=5000)
#         clf.fit(X_train_selected_with_age_sex, y_train_selected,  sample_weight=sample_weights)
#         pred_y = clf.predict(X_test_selected_with_age_sex)
#         return num_P, num_S, nFeatures, classifier, weighting, pred_y, y_test_selected

In [51]:
def calculateCorrectlyClassifiedSubjects(y_pred, y_act, test_indexes, subtype, num_P, num_S):
    """Function to calculate the number of correctly classified instances.
    
    Arguments
    ---------
        y_pred: numpy.array
            Array of predicted results.
        y_act: numpy.array
            Array of actual results.
        test_indexes: list
            List of test indexes.
        subtype: int
            The subtype.
        num_P: int
            Number of Progressor subtypes.
        num_S: int
            Number of Stable subtypes.
        weighting: int
            Weighting value for classifier.
        
    Returns
    -------
        count: int
            Number of correctly classified instances.
        index_of_intances_correctly_classified: list
            Indexes of intances correctly classified.
    """
    count = 0
    
    n_class = num_P + num_S
    
    P_range, S_range = np.arange(0,num_P), np.arange(num_P, n_class)  
    
    index_of_intances_correctly_classified = []
    
    if subtype in P_range:
        for i in range(len(y_pred)):
            # (1=stable, 2=progressor)
            if y_pred[i] == subtype and y_act[i] == 2:
                count += 1
                index_of_intances_correctly_classified.append(test_indexes[i])
    
    else:
        for i in range(len(y_pred)):
            if y_pred[i] == subtype and y_act[i] == 1:
                count += 1 
                index_of_intances_correctly_classified.append(test_indexes[i])
                
    return count, index_of_intances_correctly_classified

In [52]:
################################################# Helper Functions to Compute PPV #################################################

def TP(y_actual, y_pred):
    TP = 0
    for i in range(len(y_actual)):
        if y_actual[i] == y_pred[i] == 1:
            TP += 1
    return TP

# False positive value


def FP(y_actual, y_pred):
    FP = 0
    for i in range(len(y_actual)):
        if y_pred[i] == 1 and y_actual[i] != y_pred[i]:
            FP += 1
    return FP

# True negative value


def TN(y_actual, y_pred):
    TN = 0
    for i in range(len(y_actual)):
        if y_actual[i] == y_pred[i] == 0:
            TN += 1
    return TN

# False negative value


def FN(y_actual, y_pred):
    FN = 0
    for i in range(len(y_actual)):
        if y_pred[i] == 0 and y_actual[i] != y_pred[i]:
            FN += 1
    return FN

# tp / (tp + fp)


def precision(y_act, y_pred):
    tp = TP(y_act, y_pred)
    fp = FP(y_act, y_pred)
    if tp + fp == 0:
        return 0
    return tp / (tp + fp)


def binarize(label, arr):
    arr_copy = arr.copy()
    for i, v in enumerate(arr_copy):
        if v == label:
            arr_copy[i] = 1
        else:
            arr_copy[i] = 0
    return arr_copy

# (1=stable, 2=progressor)


def binarizeP(arr):
    arr_copy = arr.copy()
    for i, v in enumerate(arr_copy):
        if v == 2:
            arr_copy[i] = 1
        else:
            arr_copy[i] = 0
    return arr_copy

# (1=stable, 2=progressor)


def binarizeS(arr):
    arr_copy = arr.copy()
    for i, v in enumerate(arr_copy):
        if v == 1:
            arr_copy[i] = 1
        else:
            arr_copy[i] = 0
    return arr_copy


# P: [0, 1, 2]  num_P = 3
# S: [3, 4, 5]  num_S = 3
# n_class = 6

def computePPV(numP, numS, y_pred, y_act):
    n_class = numP + numS
    P_range, S_range = np.arange(0, numP), np.arange(numP, n_class)
    dic = {}
    # print("y_act",y_act)
    for i in range(n_class):
        # print("class label:", i)
        if (i in P_range):
            # print("%d in P class" %(i))
            y_p = binarize(i, y_pred)
            # print("predicted y after binarization", y_p)
            y_a = binarizeP(y_act)
            # print("actual y after binarization of P", y_a)
            score = precision(y_a, y_p)
        else:
            # print("%d in S class" %(i))
            y_p = binarize(i, y_pred)
            # print("predicted y after binarization", y_p)
            y_a = binarizeS(y_act)
            # print("actual y after binarization of S", y_a)
            score = precision(y_a, y_p)
        dic[i] = score
    return dic

def computeTP(numP, numS, subtype, y_pred, y_act, test_index):
    n_class = numP + numS
    P_range, S_range = np.arange(0, numP), np.arange(numP, n_class)
    if (subtype in P_range):
        y_p = binarize(subtype, y_pred)
        y_a = binarizeP(y_act)
        TP, FP = 0,0
        TP_index, FP_index = [], []
        for i in range(len(y_a)):
            if y_a[i] == y_p[i] == 1:
                TP += 1
                TP_index.append(test_index[i])
        for i in range(len(y_a)):
            if y_p[i] == 1 and y_a[i] != y_p[i]:
                FP += 1
                FP_index.append(test_index[i])
        return TP_index, FP_index
    
    else:
#         (subtype in S_range):
        y_p = binarize(subtype, y_pred)
        y_a = binarizeS(y_act)
        TP, FP = 0,0
        TP_index, FP_index = [], []
        for i in range(len(y_a)):
            if y_a[i] == y_p[i] == 1:
                TP += 1
                TP_index.append(test_index[i])
        for i in range(len(y_a)):
            if y_p[i] == 1 and y_a[i] != y_p[i]:
                FP += 1
                FP_index.append(test_index[i])
        return TP_index, FP_index     

In [53]:
# train_index = outer_train_index[outer]
# test_index = outer_test_index[outer]

X_train = X_data[train_scan_index]
y_train = Y[train_scan_index]
X_test = X_data[test_scan_index]
y_test = Y[test_scan_index]

print(X_train.shape)

(515, 56956)


In [54]:
print(X_train)
print(y_train)

[[-0.062971 -0.21336  -0.2084   ...  0.713     0.5       1.      ]
 [ 0.18076   0.065763  0.21935  ...  0.716     0.5       1.      ]
 [ 0.16062   0.23097   0.24299  ...  0.718     0.5       1.      ]
 ...
 [-0.14228   0.23516   0.78787  ...  0.72789   0.5       6.      ]
 [ 1.3441    0.99949   1.4091   ...  0.68846   1.        5.      ]
 [ 0.83527   0.35159   0.99831  ...  0.78085   1.        5.      ]]
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 1. 1. 1

In [55]:
row = ['PPV of interested subtype',
                    'Interested subtype',
                    'Index of intances correctly classified of interested type',
                    'Index of instances incorrectly classified into the interested type',
                    'Number of correctly classified instances',
                    'Number of Progressor Subtypes', 
                    'Number of Stable Subtypes', 
                    'Number of Features', 
                    'Classifier', 
                    'Weighting',
                    'pred_y',
                    'act_y',
                    'test_index',
                    'PPVs'
          ]

result_df = pd.DataFrame(columns = row)

In [56]:
good_analysis = []
for i in range(filtered_good_results.shape[0]):
    interested_subtype = filtered_good_results.iloc[i]['Class with High PPV']
    num_S = filtered_good_results.iloc[i]['Number of Stable Subtypes']
    num_P = filtered_good_results.iloc[i]['Number of Progressor Subtypes']
    nFeatures = filtered_good_results.iloc[i]['Number of Features']
    Classifier = filtered_good_results.iloc[i]['Classifier']
    print("nFeatures ", nFeatures)
    convert = lambda x: 0 if (x=="SVM") else 1 if (x=="RF") else 2
    
    classifier = convert(Classifier)
    weighting = filtered_good_results.iloc[i]['Weighting']
    column_indices = filtered_good_results.iloc[i]['Features selected with highest frequencies across all folds']
    #print("column_indices len ", len(column_indices))
#     print("num_S", num_S)
#     print("num_P", num_P)
#     print("nFeatures", nFeatures)
#     print("Classifier", Classifier)
#     print("classifier", classifier)
#     print("weighting", weighting)
#     print("column_indices", column_indices)
    #print("interested subtype", interested_subtype)
#     print(X_train.shape)
    try:
        num_P, num_S, nFeatures, classifier, weighting, pred_y, y_test_selected = apply(X_train, y_train, X_test, y_test, num_P, num_S, nFeatures, classifier, weighting, column_indices, dataset)
    except Exception as err:
        print('exception catched')
        continue
    print('continued')
    ppvs = computePPV(num_P, num_S, pred_y, y_test_selected)
    count, index_of_intances_correctly_classified = calculateCorrectlyClassifiedSubjects(pred_y, 
                                                                                             y_test_selected, 
                                                                                             test_scan_index, 
                                                                                             interested_subtype, 
                                                                                             num_P, 
                                                                                             num_S
                                        
                                                                                        
                                                                                        
                                                                                        )
   # print("count", count)
    #print("index_of_intances_correctly_classified", index_of_intances_correctly_classified)    
    
    
    TP_indexes, FP_indexes = computeTP(num_P, num_S, interested_subtype, pred_y, y_test_selected, test_scan_index)
    #print("FP_indexes", FP_indexes)
    print(ppvs)
    ppv_target_subtype = ppvs.get(interested_subtype)
    if ppv_target_subtype > 0.9:
        print("found good one!!! ", i)
        good_analysis.append(i)
    row = {'PPV of interested subtype': ppv_target_subtype, 
                    'Interested subtype': interested_subtype,
                    'Index of intances correctly classified of interested type': index_of_intances_correctly_classified,
                    'Index of instances incorrectly classified into the interested type': FP_indexes,
                    'Number of correctly classified instances': count,
                    'Number of Progressor Subtypes': num_P, 
                    'Number of Stable Subtypes': num_S, 
                    'Number of Features': nFeatures, 
                    'Classifier': classifier, 
                    'Weighting': weighting,
                    'pred_y': pred_y,
                    'act_y': y_test_selected,
                    'test_index': test_scan_index,
                    'PPVs': ppvs
                }

    result_df = result_df.append(row, ignore_index=True)
    

nFeatures  13


/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


continued
{0: 0.7058823529411765, 1: 0, 2: 0, 3: 0, 4: 0.4625, 5: 0, 6: 0, 7: 0}
nFeatures  30


/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


continued
{0: 0.5555555555555556, 1: 0, 2: 0, 3: 0, 4: 0.43037974683544306, 5: 0, 6: 0, 7: 0}
nFeatures  37


...

continued
{0: 0.5333333333333333, 1: 0, 2: 0, 3: 0.4268292682926829, 4: 0, 5: 0, 6: 0}
nFeatures  37


/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


continued
{0: 0.5263157894736842, 1: 0, 2: 0, 3: 0.4230769230769231, 4: 0, 5: 0, 6: 0}
nFeatures  32


/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/opt/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


continued
{0: 0.5882352941176471, 1: 0, 2: 0, 3: 0.4375, 4: 0, 5: 0, 6: 0}


In [57]:
result_df.to_csv("./X"+dataset+"_results/results/fold" + fold + "/validation.csv")
result_df.to_pickle("./X"+dataset+"_results/results/fold" + fold  + "/validation.pkl")

In [58]:
good_analysises = pd.DataFrame(good_analysis)
good_analysises.to_csv("./X"+dataset+"_results/results/fold" + fold + "/good_analysis.csv")

In [59]:
print(filtered_good_results.info)

<bound method DataFrame.info of      PPV Subjects Correctly Classified Number of Progressor Subtypes  \
0    1.0                             5                             4   
1    1.0                             5                             4   
2    1.0                             4                             4   
3    1.0                             5                             4   
4    1.0                             2                             3   
..   ...                           ...                           ...   
204  1.0                             5                             3   
205  1.0                             5                             3   
206  1.0                             5                             3   
207  1.0                             4                             3   
208  1.0                             5                             3   

    Number of Stable Subtypes Number of Features Classifier Weighting  \
0                           4 